- Test the functions of get cpmm arbs
- Get a graph representation of the pools

In [5]:

import get_cpmm_arb_txn as carb
import torch

def test_cpmm_arb():
    carb.get_cpmm_data('./one_day_arb.csv')
    carb.generate_graph_coo('./one_day_arb.csv','../arb_data/pool_info.csv')
    carb.generate_graph_input('./one_day_arb.csv','../arb_data/pools.csv',16086270)
    carb.generate_graph_output('./one_day_arb.csv','../arb_data/pools.csv',16090709,30)

# COO format of graph, represent the edges
def get_edge_index():
    graph_coo=carb.generate_graph_coo('./one_day_arb.csv','./pool_info.csv')
    return torch.tensor(graph_coo,dtype=torch.long)

# get node features x (input)
def get_x(blocknumber):
    x=carb.generate_graph_input('./one_day_arb.csv','./pools.csv',blocknumber)
    return torch.tensor(x,dtype=torch.float)

# get node targets y (output)
def get_y(blocknumber, period=50):
    y=carb.generate_graph_output('./one_day_arb.csv','./pools.csv',blocknumber,period)
    return torch.tensor(y,dtype=torch.long)


- Test of a PyG `Data` object

In [7]:
from torch_geometric.data import Data

graph_edges=get_edge_index()

blocknumber=16086270
data = Data(edge_index=graph_edges,x=get_x(blocknumber),y=get_y(blocknumber))

print(data)

generating graph COO...
number of nodes:489


100%|██████████| 489/489 [03:25<00:00,  2.38it/s]


number of edges in graph: 156
generating graph input...


100%|██████████| 489/489 [00:00<00:00, 39126.57it/s]


shape of the graph input signal:  (489, 630)
[[0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 3.85691961e-20]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.52986602e-14]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.59581835e-13]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 2.86897843e-10]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 1.05925984e-09]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 8.92902469e-14]]
generating graph output...
Data(x=[489, 630], edge_index=[2, 156], y=[489, 1])


- Create my own dataset
- The key step is the method `process`
- In `process`, the raw data is parsed into a list of PyG `Data` objects

In [14]:
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

        # load data from the save path using `process`
        self.data, self.slices = torch.load(self.processed_paths[0])

    # return raw data file names
    @property
    def raw_file_names(self):
        return ['./one_day_arb.csv']

    # return the name when saving the dataset files with method `process`
    @property
    def processed_file_names(self):
        return ['data.pt']
    
    # no need, this is for downloading dataset from the web
    # def download(self):
    #     # Download to `self.raw_dir`.
    #     download_url(url, self.raw_dir)
    #     ...

    # methods to create the dataset
    def process(self):
        
        # Read data into huge PyG `Data` list.
        start_block_num = 16086234
        end_block_num = 16093396
        duration = 50

        data_list = []

        graph_edges=get_edge_index()

        for blocknumber in tqdm(range(start_block_num,end_block_num)):
            data = Data(edge_index=graph_edges,x=get_x(blocknumber),y=get_y(blocknumber,duration))
            data_list.append(data)
            blocknumber+=duration

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

dataset=MyOwnDataset('dataset')
# process data and save
dataset.process()
print('process finished!')

Processing...


generating graph COO...
number of nodes:489


100%|██████████| 489/489 [03:24<00:00,  2.40it/s]


number of edges in graph: 156


TypeError: 'module' object is not callable